### Задание №1.
В коде из методички реализуйте один или несколько критериев останова: минимальное количество объектов в листе (min_leaf), максимальная глубина дерева, максимальное количество листьев и т.д. Добавьте эти критерии в параметры функции build_tree и проверьте ее работоспособность.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

In [3]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.classes = self.get_classes()
        self.prediction = self.predict()
    
    def get_classes(self):
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        return classes
        
    def predict(self):    
        prediction = max(self.classes, key=self.classes.get)
        return prediction

In [4]:
class ColorText:
    PURPLE = '\033[1;35;48m'
    CYAN = '\033[1;36;48m'
    BOLD = '\033[1;39;48m'
    GREEN = '\033[1;34;48m'
    BLUE = '\033[1;44;48m'
    ORANGE = '\033[1;32;48m'
    YELLOW = '\033[1;33;48m'
    RED = '\033[1;31;48m'
    BLACK = '\033[1;30;48m'
    UNDERLINE = '\033[1;37;48m'
    END = '\033[1;37;0m'

In [5]:
def get_inform_index(labels, type_index='gini'):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    if type == 'gini':
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2
    else:
        impurity = 0
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p * np.log2(p) 
        
    return impurity

In [6]:
def quality(left_labels, right_labels, type_index, current_index):

    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_index - p * get_inform_index(left_labels, type_index) - (1 - p) * get_inform_index(right_labels, type_index)

In [7]:
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
def find_best_split(data, labels, min_leaf, inform_index):
    current_index = get_inform_index(labels, inform_index)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, inform_index, current_index)
            
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [9]:
def build_tree(data, labels, min_leaf=5, max_depth=None, inform_index='gini'):

    quality, t, index = find_best_split(data, labels, min_leaf, inform_index)

    if quality == 0 or max_depth == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    if max_depth:
        true_branch = build_tree(true_data, true_labels, min_leaf, max_depth - 1)
        false_branch = build_tree(false_data, false_labels, min_leaf, max_depth - 1)
    else:
        true_branch = build_tree(true_data, true_labels, min_leaf)
        false_branch = build_tree(false_data, false_labels, min_leaf)
        
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):
    
    preds = []
    for obj in data:
        prediction = classify_object(obj, tree)
        preds.append(prediction)
    return np.array(preds)

In [12]:
def accuracy_metric(real, pred):
    return np.sum(real == pred) / real.shape[0]

In [13]:
def print_tree(node, spacing=""):

    if isinstance(node, Leaf):
        print(ColorText.ORANGE + spacing + ' ЛИСТ' 
                  + ': прогноз = ' + str(node.prediction) 
                  + ', объектов = ' + str(len(node.labels)) 
                  + ', классы: ' + str(node.classes)
                  + ColorText.END)
        return

    print(ColorText.GREEN + spacing + 'УЗЕЛ'  
              + ': индекс = ' + str(node.index) 
              + ', порог = ' + str(round(node.t, 2))
              + ColorText.END)

    print (spacing + '--> Левая ветка:')
    print_tree(node.true_branch, spacing + "   ")

    print (spacing + '--> Правая ветка:')
    print_tree(node.false_branch, spacing + "   ")

Сгенерируем датасет с двумя информативными признаками и двумя классами из 1000 объектов: 

In [14]:
classification_data, classification_labels = datasets.make_classification(n_samples=1000, 
                                                      n_features=2, n_informative=2, 
                                                      n_classes=2, n_redundant=0, 
                                                      n_clusters_per_class=1, 
                                                      random_state=5)

In [15]:
train_data, test_data, train_labels, test_labels = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size = 0.3,
                                                                    random_state = 1)

- Построим дерево решений для классификации объектов с минимальным количеством объектов в листе равным 10 без ограничения глубины дерева:

In [16]:
my_tree = build_tree(train_data, train_labels, min_leaf=10)

In [17]:
print_tree(my_tree)

УЗЕЛ: индекс = 0, порог = -0.0
--> Левая ветка:
   УЗЕЛ: индекс = 1, порог = -1.4
   --> Левая ветка:
      УЗЕЛ: индекс = 1, порог = -1.66
      --> Левая ветка:
         УЗЕЛ: индекс = 0, порог = -0.8
         --> Левая ветка:
             ЛИСТ: прогноз = 0, объектов = 10, классы: {0: 7, 1: 3}
         --> Правая ветка:
             ЛИСТ: прогноз = 0, объектов = 52, классы: {0: 52}
      --> Правая ветка:
         УЗЕЛ: индекс = 0, порог = -0.69
         --> Левая ветка:
             ЛИСТ: прогноз = 0, объектов = 13, классы: {0: 13}
         --> Правая ветка:
            УЗЕЛ: индекс = 0, порог = -0.4
            --> Левая ветка:
               УЗЕЛ: индекс = 1, порог = -1.59
               --> Левая ветка:
                   ЛИСТ: прогноз = 1, объектов = 11, классы: {0: 3, 1: 8}
               --> Правая ветка:
                   ЛИСТ: прогноз = 0, объектов = 10, классы: {0: 8, 1: 2}
            --> Правая ветка:
               УЗЕЛ: индекс = 1, порог = -1.52
               --> Лева

In [18]:
train_answers = predict(train_data, my_tree)
test_answers = predict(test_data, my_tree)

Получим следующие метрики accuracy для обучающей и тестовой выборки соответственно:

In [19]:
accuracy_metric(train_labels, train_answers)

0.9785714285714285

In [20]:
accuracy_metric(test_labels, test_answers)

0.9433333333333334

- А теперь построим дерево решений с тем же минимальным количеством объектов в листе равным 10, но с ограничением глубины дерева равным 3:

In [21]:
my_tree_1 = build_tree(train_data, train_labels, min_leaf=10, max_depth=3)

In [22]:
print_tree(my_tree_1)

УЗЕЛ: индекс = 0, порог = -0.0
--> Левая ветка:
   УЗЕЛ: индекс = 1, порог = -1.4
   --> Левая ветка:
      УЗЕЛ: индекс = 1, порог = -1.66
      --> Левая ветка:
          ЛИСТ: прогноз = 0, объектов = 62, классы: {0: 59, 1: 3}
      --> Правая ветка:
          ЛИСТ: прогноз = 1, объектов = 61, классы: {0: 28, 1: 33}
   --> Правая ветка:
       ЛИСТ: прогноз = 0, объектов = 211, классы: {0: 211}
--> Правая ветка:
   УЗЕЛ: индекс = 1, порог = -1.45
   --> Левая ветка:
       ЛИСТ: прогноз = 0, объектов = 37, классы: {0: 37}
   --> Правая ветка:
      УЗЕЛ: индекс = 0, порог = 1.2
      --> Левая ветка:
          ЛИСТ: прогноз = 1, объектов = 184, классы: {1: 181, 0: 3}
      --> Правая ветка:
          ЛИСТ: прогноз = 1, объектов = 145, классы: {1: 145}


In [23]:
train_answers_1 = predict(train_data, my_tree_1)
test_answers_1 = predict(test_data, my_tree_1)

Получим следующие метрики accuracy для обучающей и тестовой выборки соответственно:

In [24]:
accuracy_metric(train_labels, train_answers_1)

0.9514285714285714

In [25]:
accuracy_metric(test_labels, test_answers_1)

0.94

Как видим, метрики для первого и второго деревьев для тестовой выборки практически не изменилась, а дерево значительно упростилось.
___

### Задание №2.
Для задачи классификации обучите дерево решений с использованием критериев разбиения Джини и Энтропия. Сравните качество классификации, сделайте выводы.

Расширил функционал индексов информативности в первом задании, добавив возможность выбрать тип индекса в функции построения дерева:

In [26]:
my_tree_gini = build_tree(train_data, train_labels, min_leaf=10, inform_index='gini')
my_tree_entropy = build_tree(train_data, train_labels, min_leaf=10, inform_index='entropy')

In [27]:
print_tree(my_tree_gini)

УЗЕЛ: индекс = 0, порог = -0.0
--> Левая ветка:
   УЗЕЛ: индекс = 1, порог = -1.4
   --> Левая ветка:
      УЗЕЛ: индекс = 1, порог = -1.66
      --> Левая ветка:
         УЗЕЛ: индекс = 0, порог = -0.8
         --> Левая ветка:
             ЛИСТ: прогноз = 0, объектов = 10, классы: {0: 7, 1: 3}
         --> Правая ветка:
             ЛИСТ: прогноз = 0, объектов = 52, классы: {0: 52}
      --> Правая ветка:
         УЗЕЛ: индекс = 0, порог = -0.69
         --> Левая ветка:
             ЛИСТ: прогноз = 0, объектов = 13, классы: {0: 13}
         --> Правая ветка:
            УЗЕЛ: индекс = 0, порог = -0.4
            --> Левая ветка:
               УЗЕЛ: индекс = 1, порог = -1.59
               --> Левая ветка:
                   ЛИСТ: прогноз = 1, объектов = 11, классы: {0: 3, 1: 8}
               --> Правая ветка:
                   ЛИСТ: прогноз = 0, объектов = 10, классы: {0: 8, 1: 2}
            --> Правая ветка:
               УЗЕЛ: индекс = 1, порог = -1.52
               --> Лева

In [28]:
print_tree(my_tree_entropy)

УЗЕЛ: индекс = 0, порог = -0.0
--> Левая ветка:
   УЗЕЛ: индекс = 1, порог = -1.4
   --> Левая ветка:
      УЗЕЛ: индекс = 1, порог = -1.66
      --> Левая ветка:
         УЗЕЛ: индекс = 0, порог = -0.8
         --> Левая ветка:
             ЛИСТ: прогноз = 0, объектов = 10, классы: {0: 7, 1: 3}
         --> Правая ветка:
             ЛИСТ: прогноз = 0, объектов = 52, классы: {0: 52}
      --> Правая ветка:
         УЗЕЛ: индекс = 0, порог = -0.69
         --> Левая ветка:
             ЛИСТ: прогноз = 0, объектов = 13, классы: {0: 13}
         --> Правая ветка:
            УЗЕЛ: индекс = 0, порог = -0.4
            --> Левая ветка:
               УЗЕЛ: индекс = 1, порог = -1.59
               --> Левая ветка:
                   ЛИСТ: прогноз = 1, объектов = 11, классы: {0: 3, 1: 8}
               --> Правая ветка:
                   ЛИСТ: прогноз = 0, объектов = 10, классы: {0: 8, 1: 2}
            --> Правая ветка:
               УЗЕЛ: индекс = 1, порог = -1.52
               --> Лева

In [29]:
test_answers_gini = predict(test_data, my_tree_gini)
test_answers_entropy = predict(test_data, my_tree_entropy)

In [30]:
accuracy_metric(test_labels, test_answers_gini)

0.9433333333333334

In [31]:
accuracy_metric(test_labels, test_answers_entropy)

0.9433333333333334

Как видим, на качество классификации индекс информативности при нахождении лучшего разбиения не повлиял.